## Equalizadores

lorem ipsum dor sit amet

In [10]:
%run -i 2_1introducao.ipynb

from math import tan, pi, cos

In [11]:
def run_filter(cutoff_frequency_int, file_name, filter_function, gain, filter_class = 'lowpass', center_frequency = 0, bandwidth = 0):
    # leitura do arquivo WAV
    sample_rate, input_signal = read_audio_file(file_name)
    # aplicação do filtro

    nyq = 0.5 * sample_rate
    normal_cutoff = cutoff_frequency_int / nyq
    center_frequency = center_frequency / nyq
    bandwidth = bandwidth / nyq

    if filter_function == 'shelving':
        filtered_data, w, h = shelving(input_signal, normal_cutoff, gain, filter_class)
    else:
        filtered_data, w, h = peak(input_signal, center_frequency, bandwidth, gain, filter_class)
  
    plot_filter(cutoff_frequency_int, sample_rate, w, h)
   
    write_audio_file(filtered_data, sample_rate, 'output_file_equalizadores.wav')

    %matplotlib ipympl
    plot_signal(input_signal, sample_rate, 'Sinal de Entrada')
    plot_signal(filtered_data, sample_rate, 'Sinal de Saída')
 

    display_audio(file_name, 'output_file_equalizadores.wav')


In [12]:

def shelving(input_signal, normal_cutoff, gain, filter_class):
    output_signal = [0] * len(input_signal)
    V0 = 10**(gain/20)
    H0 = V0 - 1
    if gain >= 0:
        c = (tan(pi*normal_cutoff/2) - 1)/(tan(pi*normal_cutoff/2)+1)
    else:
        c = (tan(pi*normal_cutoff/2) - V0)/(tan(pi*normal_cutoff/2)+V0)
    xh = 0
    for n in range(len(input_signal)):
        xh_new = input_signal[n] - c*xh
        ap_y = c * xh_new + xh
        xh = xh_new
        if filter_class == 'lowpass':
            output_signal[n] = 0.5 * H0 * (input_signal[n] + ap_y) + input_signal[n]
        else:
            output_signal[n] = 0.5 * H0 * (input_signal[n] - ap_y) + input_signal[n]
    w, h = freqz(output_signal, input_signal, fs=44100)

    return output_signal, w, h


def peak(input_signal, center_frequency, bandwidth, gain, filter_class):
    output_signal = [0] * len(input_signal) 
    V0 = 10**(gain/20)
    H0 = V0 - 1
    if gain >= 0:
        c = (tan(pi*bandwidth/2) - 1)/(tan(pi*bandwidth/2)+1)
    else:
        c = (tan(pi*bandwidth/2) - V0)/(tan(pi*bandwidth/2)+V0)
    d = -cos(pi*center_frequency)
    xh = [0,0]
    for n in range(len(input_signal)):
        xh_new = input_signal[n] - d*(1-c)*xh[0] + c*xh[1]
        ap_y = -c * xh_new + d*(1-c)*xh[0] + xh[1]
        xh = [xh_new, xh[0]]
        if filter_class == 'lowpass':
            output_signal[n] = 0.5 * H0 * (input_signal[n] + ap_y) + input_signal[n]
        else:
            output_signal[n] = 0.5 * H0 * (input_signal[n] - ap_y) + input_signal[n]
    w, h = freqz(output_signal, input_signal, fs=44100)
    magnitude = np.abs(h)
    limiar = 1
    indices_superior = np.where(magnitude > limiar)[0]
    indices_inferior = np.where(magnitude[::-1] > limiar)[0]
    frequencia_superior = w[indices_superior[0]]
    frequencia_inferior = w[::-1][indices_inferior[0]]
    largura_banda = frequencia_superior - frequencia_inferior
    center = largura_banda/2
    print("center frequency:", abs(center))
    return output_signal, w, h


#rodar o filtro
clear_output()
out1.clear_output()
out2.clear_output()
out3.clear_output()
run_filter(5000, 'symphony.wav', 'shelving', 6, 'lowpass')
widgets_equalizers()

Dropdown(description='Filter class:', index=1, options=('lowpass', 'highpass'), value='highpass')

Dropdown(description='Filter type:', options=('shelving', 'peak'), value='shelving')

Text(value='symphony.wav', continuous_update=False, description='File name')

Text(value='6', continuous_update=False, description='Gain')

Button(button_style='success', description='Atualizar', style=ButtonStyle())

Text(value='5000', continuous_update=False, description='Cutoff frequency')

Button(button_style='success', description='Atualizar', style=ButtonStyle())